This notebook shows how to compute an instance Norm (+ add + relu) using the cuDNN python frontend.

$$y = \text{InstanceNorm(x)} = \frac{x - \mathbb{E}(x)}{\sqrt{Var(x)+\epsilon}}\cdot \gamma + \beta$$
$$\text{ReLU}(y + \alpha)$$

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NVIDIA/cudnn-frontend/blob/main/samples/python/01_matmul_bias.ipynb)

## Prerequisites and Setup
This notebook requires an NVIDIA GPU. If `nvidia-smi` fails, go to Runtime -> Change runtime type -> Hardware accelerator and confirm a GPU is selected.

In [ ]:
# get_ipython().system('nvidia-smi')

If running on Colab, you will need to install the cudnn python interface.

In [ ]:
# get_ipython().system('pip install nvidia-cudnn-cu12')
# get_ipython().system('pip install nvidia-cudnn-frontend')
# get_ipython().system('pip3 install --pre torch --index-url https://download.pytorch.org/whl/nightly/cu128')

#### General Setup
The cudnn handle is a per device handle used to initialize cudnn context.



In [ ]:
import cudnn
import torch
import sys

torch.manual_seed(0)
handle = cudnn.create_handle()

print("Running with cudnn backend version:", cudnn.backend_version())

assert torch.cuda.is_available()

### InstanceNorm Reference Computation


#### Problem Sizes


In [ ]:
N, C, H, W = 16, 32, 64, 64

input_type = torch.float16

# Epsilon is a small number to prevent division by 0.
epsilon_value = 1e-5

Create input tensor GPU buffers. We use PyTorch to allocate GPU tensors so we can reuse them easily when we calculate reference outputs.

In [ ]:
# input tensor memory, initialize them to random numbers
x_gpu = torch.randn(
    (N, C, H, W), dtype=input_type, requires_grad=True, device="cuda"
).to(memory_format=torch.channels_last)

scale_gpu = torch.randn(
    (1, C, 1, 1), dtype=input_type, requires_grad=True, device="cuda"
).to(memory_format=torch.channels_last)

bias_gpu = torch.randn(
    (1, C, 1, 1), dtype=input_type, requires_grad=True, device="cuda"
).to(memory_format=torch.channels_last)

# set epsilon to epsilon_value, allocate on cpu.
epsilon_cpu = torch.full(
    (1, 1, 1, 1), epsilon_value, dtype=torch.float32, requires_grad=False, device="cpu"
)

# Create tensors for fusion and intermediate ops
a_gpu = torch.randn_like(x_gpu)
sum_gpu = torch.empty_like(a_gpu)  # Needs to have the same datatype!

Compute reference ouputs and allocate output tensor GPU buffers

In [ ]:
# we create the reference computation outputs here so we can use .empty_like() to create our output buffers
y_expected = torch.nn.functional.instance_norm(
    x_gpu, weight=scale_gpu.view(C), bias=bias_gpu.view(C)
)

mean_expected = x_gpu.to(torch.float32).mean(dim=(2, 3), keepdim=True)

inv_var_expected = torch.rsqrt(
    torch.var(x_gpu.to(torch.float32), dim=(2, 3), keepdim=True) + epsilon_value
)

sum_expected = y_expected + a_gpu

relu_expected = torch.nn.functional.relu(sum_expected)

mask_expected = relu_expected > 0.0

# allocate output tensor memory using PyTorch
# PyTorch has calculated their shapes already, so we can simply use .empty_like()

# Comparing x_gpu and out_expected. The types are the same, the dimensions are the same.
# The only difference is grad_fn=<ToCopyBackward0> (x_gpu) vs grad_fn=<ViewBackward0> (out_expected)
y_gpu = torch.empty_like(x_gpu)
mean_gpu = torch.empty_like(mean_expected)
inv_var_gpu = torch.empty_like(inv_var_expected)
relu_gpu = torch.empty_like(a_gpu)  # Relu preserves size and datatype?

zero_gpu = torch.full(
    (1, 1, 1, 1), 0.0, dtype=torch.float16, requires_grad=False, device="cuda"
)
mask_gpu = torch.zeros(
    x_gpu.size(), dtype=torch.bool, layout=x_gpu.layout, device="cuda"
)

#### Create cuDNN graph and tensors

In [ ]:
# Create the graph
graph = cudnn.pygraph(
    handle=handle,
    intermediate_data_type=cudnn.data_type.FLOAT,
    compute_data_type=cudnn.data_type.FLOAT,
)

# create tensor handles with the graph API
x = graph.tensor_like(x_gpu.detach()).set_name("X")
scale = graph.tensor_like(scale_gpu.detach()).set_name("scale")
bias = graph.tensor_like(bias_gpu.detach()).set_name("bias")
epsilon = graph.tensor_like(epsilon_cpu).set_name("epsilon")
(y, mean, inv_var) = graph.instancenorm(
    name="instancenorm",
    input=x,
    norm_forward_phase=cudnn.norm_forward_phase.INFERENCE,  # cudnn.norm_forward_phase.TRAINING
    scale=scale,
    bias=bias,
    epsilon=epsilon,
)


# Add an addition operation between the instance norm output and some other tensor 'A'.
a = graph.tensor_like(x).set_name("A")

sum = graph.add(y, a, name="add")

sum.set_name("sum")

# Add a relu operation on the result of the addition
relu = graph.relu(sum)

relu.set_name("sum").set_output(True).set_data_type(cudnn.data_type.HALF)
zero = graph.tensor_like(epsilon).set_name("zero").set_data_type(cudnn.data_type.HALF)


mask = graph.cmp_gt(input=relu, comparison=zero, name="mask")

mask.set_output(True).set_name("mask").set_data_type(cudnn.data_type.BOOLEAN)

#### Build the graph

In [ ]:
# Build the graph
graph.build([cudnn.heur_mode.A, cudnn.heur_mode.FALLBACK])

# To run this block more than once, we need to re-run the previous block to get a new graph.
# The same instance of a graph should not be built twice.

#### Execute the graph

In [ ]:
# Mapping of (handles -> memory)
variant_pack = {
    x: x_gpu.detach(),
    scale: scale_gpu.detach(),
    bias: bias_gpu.detach(),
    epsilon: epsilon_cpu,
    a: a_gpu,
    zero: zero_gpu,
    mask: mask_gpu,
    relu: relu_gpu,
    mean: mean_gpu,
    inv_var: inv_var_gpu,
}

workspace = torch.empty(graph.get_workspace_size(), device="cuda", dtype=torch.uint8)
graph.execute(variant_pack, workspace)
torch.cuda.synchronize()

Test cuDNN's output against PyTorch's and check correctness

In [ ]:
# reference output
torch.testing.assert_close(relu_gpu, relu_expected, rtol=5e-3, atol=5e-3)
# torch.testing.assert_close(mask_gpu, mask_expected)

Perform Cleanup

In [ ]:
cudnn.destroy_handle(handle)